In [1]:
import pandas as pd
import pylogit
import biogeme
from statsmodels.formula.api import logit
import numpy as np

In [2]:
sacog = pd.read_excel('sacog_recoded_for_qianhua.xlsx')

In [3]:
# Solve inconsistency in data coding and save as new dataset
sacog = sacog.replace('Never ', 'Never')
sacog.to_excel('sacog_recoded_012021.xlsx')

In [4]:
never = ["Never", "Almost never"]
never_and_noRes = ["Never", "Almost never", 'No Response']

def count_(cat):
    past_year_freq = 'PastYear-ECommerce-Frequency-' + cat
    may_freq = 'May-ECommerce-Frequency-' + cat
    may_order = 'May-ECommerce-Orders-' + cat
    oldUser = sacog[~sacog[past_year_freq].isin(never_and_noRes)]
    pastNever = sacog[sacog[past_year_freq].isin(never)]
    adopter = pastNever[(pastNever[may_order] > 0) | (~pastNever[may_freq].isin(never_and_noRes))]
    NonAdopter = pastNever[(pastNever[may_order] == 0) & pastNever[may_freq].isin(never)]
    quiter = oldUser[(oldUser[may_order] == 0) & (oldUser[may_freq].isin(never))]
    keeper = oldUser[(oldUser[may_order] > 0) | (~oldUser[may_freq].isin(never_and_noRes))]
    nowUser = sacog[(sacog[may_order] > 0) | (~sacog[may_freq].isin(never_and_noRes))]
    return [len(oldUser),
           len(pastNever), 
           len(adopter),
           len(NonAdopter),
           len(quiter),
           len(keeper),
           len(nowUser),
           oldUser,
           pastNever,
           adopter,
           NonAdopter,
           quiter,
           keeper,
           nowUser
           ]

dict_ = {}
cat_ = ['PreparedFood', 'Groceries', 'OtherFood', 'PaperCleaning', 'Clothing', 'HomeOffice', 'Medication', 'ChildcareItems', 'Other']
for cat in cat_:
    dict_[cat] = count_(cat)

In [5]:
def adoption(row, cat):
    may_freq = 'May-ECommerce-Frequency-' + cat
    may_order = 'May-ECommerce-Orders-' + cat
    if (row[may_order] > 0) | (row[may_freq] not in never_and_noRes):
        return 1 # Adopt/keeper
    elif (row[may_order] == 0) & (row[may_freq] in never):
        return 0 # Not Adopt/quiter
    else:
        return 999 # can't identify 

## 1) E-prepared-food

In [6]:
pastNeverF = count_('PreparedFood')[8]
oldUserF = count_('PreparedFood')[7]

In [7]:
pastNeverF['adopt'] = pastNeverF.apply(lambda row: adoption(row, 'PreparedFood'), axis = 1)
oldUserF['quit'] = oldUserF.apply(lambda row: adoption(row, 'PreparedFood'), axis = 1)

/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
adoptF = pastNeverF[pastNeverF['adopt'] != 999][['adopt', 'COVID-PPEReducesContraction', 'COVID-PPEReducesTransmission', 
                  'COVID-AbleSocialDistance', 'Trust-FoodDeliveryCompanies',
                  'Trust-FoodDeliveryDrivers', 'Behavior-PickupExcuseToGetOut',
                  'Behavior-NotMindRestaurantOnline', 'ECommerce-Food-HardToKnowExactly',
                  'ECommerce-Food-WorryAboutTheft', 'Attitude-SociallyIsolated',
                  'Attitude-Stressed', 'Attitude-HaveEnoughFood', 'Online-Ease']]

In [9]:
# Remove record with response -9998 or 997, reduce to 89 records
adoptF = adoptF[~np.isin(sampleF, [-9998, 997]).any(axis=1)]

In [10]:
sampleF.describe()

,adopt,COVID-PPEReducesContraction,COVID-PPEReducesTransmission,COVID-AbleSocialDistance,Trust-FoodDeliveryCompanies,Trust-FoodDeliveryDrivers,Behavior-PickupExcuseToGetOut,Behavior-NotMindRestaurantOnline,ECommerce-Food-HardToKnowExactly,ECommerce-Food-WorryAboutTheft,Attitude-SociallyIsolated,Attitude-Stressed,Attitude-HaveEnoughFood,Online-Ease
count,89.000000,89.000000,89.000000,89.00000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000
mean,0.550562,3.966292,4.359551,4.41573,3.258427,3.629213,2.269663,3.146067,3.932584,2.640449,2.865169,2.988764,4.550562,4.101124
std,0.500255,1.210273,1.014074,0.61808,0.885849,0.728978,1.105400,1.092854,0.876431,1.100074,1.109781,1.123048,0.522477,0.798364
min,0.000000,1.000000,1.000000,2.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000
25%,0.000000,3.000000,4.000000,4.00000,3.000000,3.000000,1.000000,2.000000,4.000000,2.000000,2.000000,2.000000,4.000000,4.000000
50%,1.000000,4.000000,5.000000,4.00000,3.000000,4.000000,2.000000,3.000000,4.000000,2.000000,3.000000,3.000000,5.000000,4.000000
75%,1.000000,5.000000,5.000000,5.00000,4.000000,4.000000,3.000000,4.000000,4.000000,3.000000,4.000000,4.000000,5.000000,5.000000
max,1.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [11]:
import statsmodels.api as sm

X = sampleF.iloc[:,1:]
Y = sampleF.iloc[:,0]
X = sm.add_constant(X) #?
est = sm.Logit(Y, X).fit()
print(est.summary())


Optimization terminated successfully.
         Current function value: 0.506493
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  adopt   No. Observations:                   89
Model:                          Logit   Df Residuals:                       75
Method:                           MLE   Df Model:                           13
Date:                Wed, 27 Jan 2021   Pseudo R-squ.:                  0.2638
Time:                        16:17:18   Log-Likelihood:                -45.078
converged:                       True   LL-Null:                       -61.234
Covariance Type:            nonrobust   LLR p-value:                  0.002159
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -1.7578      3.471     -0.50

/Users/apple/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


## 2) E-grocery

In [12]:
pastNeverG = count_('Groceries')[8]

In [13]:
pastNeverG['adopt'] = pastNeverG.apply(lambda row: adoption(row, 'Groceries'), axis = 1)

/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
sampleG = pastNeverG[pastNeverG['adopt'] != 999][['adopt', 'COVID-PPEReducesContraction', 'COVID-PPEReducesTransmission', 
                  'COVID-AbleSocialDistance', 'Trust-GroceryDeliveryCompanies',
                  'Trust-GroceryDeliveryDrivers', 'Behavior-GroceryExcuseToGetOut',
                  'Behavior-NotMindGroceriesOnline', 'Behavior-InPersonIsChore', 
                  'ECommerce-Food-HardToKnowExactly', 'ECommerce-Food-WorryAboutTheft', 
                  'Attitude-SociallyIsolated', 'Attitude-Stressed', 
                  'Attitude-HaveEnoughFood', 'Online-Ease']]

In [15]:
# Remove record with response -9998 or 997, reduce to 87 records
sampleG = sampleG[~np.isin(sampleG, [-9998, 997]).any(axis=1)]

In [16]:
X = sampleG.iloc[:,1:]
Y = sampleG.iloc[:,0]
X = sm.add_constant(X) #?
est = sm.Logit(Y, X).fit()
print(est.summary())

Optimization terminated successfully.
         Current function value: 0.464399
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  adopt   No. Observations:                  150
Model:                          Logit   Df Residuals:                      135
Method:                           MLE   Df Model:                           14
Date:                Wed, 27 Jan 2021   Pseudo R-squ.:                  0.2755
Time:                        16:17:18   Log-Likelihood:                -69.660
converged:                       True   LL-Null:                       -96.155
Covariance Type:            nonrobust   LLR p-value:                 1.909e-06
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -5.2241      3.151     -1.65

/Users/apple/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
